In [30]:
import requests
import requests_cache
import networkx as nx
import pandas as pd

In [2]:
from fake_useragent import UserAgent
from functools import partial, wraps

In [3]:
requests_cache.install_cache(
    'nba-rank',
    expire_after=24*60*60,
    backend='sqlite'
)

In [4]:
def apply_to_output(callback, *cb_args, **cb_kwargs):

    def decorator(func):

        @wraps(func)
        def new_func(*args, **kwargs):
            return callback(func(*args, **kwargs), *cb_args, **cb_kwargs)

        return new_func

    return decorator

In [5]:
def get_json(url, *args, **kwargs):
    r = requests.get(url=url, *args, **kwargs)
    r.raise_for_status()
    return r.json()

In [6]:
ua = UserAgent()

In [7]:
get_json = partial(get_json, headers={'User-Agent': ua.google})

In [8]:
def nested_dict_from_lst(lst, key):
    return {d.pop(key): d for d in lst}

In [9]:
def normalize_dict(dct, frame_name_key='name', frames_key='resultSets',
                   frames_default=[]):
    return nested_dict_from_lst(dct.get(frames_key, frames_default),
                                key=frame_name_key)

In [10]:
def to_data_frame(frames_dict, frame_name, frame_index=None,
                  frame_data_key='rowSet', frame_columns_key='headers'):
    return pd.DataFrame.from_records(
               data=frames_dict[frame_name][frame_data_key],
               columns=frames_dict[frame_name][frame_columns_key],
               index=frame_index).dropna()

In [12]:
@apply_to_output(to_data_frame, frame_name='LeagueGameLog', frame_index=['GAME_ID'])
@apply_to_output(normalize_dict)
def games(league_id='00', season='2015-16', season_type='Regular Season',
          team=True, sort_by='date', ascending=True):

    return get_json(url='http://stats.nba.com/stats/LeagueGameLog',
                    params={'LeagueID': league_id,
                            'PlayerOrTeam': 'T' if team else 'P',
                            'Season': season,
                            'SeasonType': season_type,
                            'Sorter': sort_by.upper(),
                            'Direction': 'ASC' if ascending else 'DESC'})

In [14]:
@apply_to_output(to_data_frame, frame_name='TeamYears', frame_index=['TEAM_ID'])
@apply_to_output(normalize_dict)
def teams(league_id='00'):

    return get_json(url='http://stats.nba.com/stats/commonTeamYears',
                    params={'LeagueID': league_id})

In [15]:
@apply_to_output(to_data_frame, frame_name='TeamInfoCommon', frame_index=['TEAM_ID'])
@apply_to_output(normalize_dict)
def team(team_id, league_id='00', season='2015-16',
         season_type='Regular Season'):

    return get_json(url='http://stats.nba.com/stats/TeamInfoCommon',
                    params={'LeagueID': league_id,
                            'TeamID': team_id,
                            'Season': season,
                            'SeasonType': season_type})

In [27]:
teams_df = teams()
teams_df

,LEAGUE_ID,MIN_YEAR,MAX_YEAR,ABBREVIATION
TEAM_ID,,,,
1610612737,00,1949,2016,ATL
1610612738,00,1946,2016,BOS
1610612739,00,1970,2016,CLE
1610612740,00,2002,2016,NOP
1610612741,00,1966,2016,CHI
1610612742,00,1980,2016,DAL
1610612743,00,1976,2016,DEN
1610612744,00,1946,2016,GSW
1610612745,00,1967,2016,HOU


In [28]:
teams_details = pd.concat(map(team, teams_df.index))
teams_details

,SEASON_YEAR,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CONFERENCE,TEAM_DIVISION,TEAM_CODE,W,L,PCT,CONF_RANK,DIV_RANK,MIN_YEAR,MAX_YEAR
TEAM_ID,,,,,,,,,,,,,,
1610612737,2015-16,Atlanta,Hawks,ATL,East,Southeast,hawks,48,34,0.585,4,2,1949,2016
1610612738,2015-16,Boston,Celtics,BOS,East,Atlantic,celtics,48,34,0.585,5,2,1946,2016
1610612739,2015-16,Cleveland,Cavaliers,CLE,East,Central,cavaliers,57,25,0.695,1,1,1970,2016
1610612740,2015-16,New Orleans,Pelicans,NOP,West,Southwest,pelicans,30,52,0.366,12,5,2002,2016
1610612741,2015-16,Chicago,Bulls,CHI,East,Central,bulls,42,40,0.512,9,4,1966,2016
1610612742,2015-16,Dallas,Mavericks,DAL,West,Southwest,mavericks,42,40,0.512,6,2,1980,2016
1610612743,2015-16,Denver,Nuggets,DEN,West,Northwest,nuggets,33,49,0.402,11,4,1976,2016
1610612744,2015-16,Golden State,Warriors,GSW,West,Pacific,warriors,73,9,0.890,1,1,1946,2016
1610612745,2015-16,Houston,Rockets,HOU,West,Southwest,rockets,41,41,0.500,8,4,1967,2016


In [31]:
G = nx.MultiDiGraph()

In [32]:
G.add_nodes_from([(1, dict(foo=42)), (5, dict(bar=21))])

In [36]:
list(G.nodes(data=True))

[(1, {'foo': 42}), (5, {'bar': 21})]

In [38]:
G.node[1]

{'foo': 42}

In [49]:
teams_details.to_dict(orient='index')

{1610612737: {'CONF_RANK': 4,
  'DIV_RANK': 2,
  'L': 34,
  'MAX_YEAR': '2016',
  'MIN_YEAR': '1949',
  'PCT': 0.585,
  'SEASON_YEAR': '2015-16',
  'TEAM_ABBREVIATION': 'ATL',
  'TEAM_CITY': 'Atlanta',
  'TEAM_CODE': 'hawks',
  'TEAM_CONFERENCE': 'East',
  'TEAM_DIVISION': 'Southeast',
  'TEAM_NAME': 'Hawks',
  'W': 48},
 1610612738: {'CONF_RANK': 5,
  'DIV_RANK': 2,
  'L': 34,
  'MAX_YEAR': '2016',
  'MIN_YEAR': '1946',
  'PCT': 0.585,
  'SEASON_YEAR': '2015-16',
  'TEAM_ABBREVIATION': 'BOS',
  'TEAM_CITY': 'Boston',
  'TEAM_CODE': 'celtics',
  'TEAM_CONFERENCE': 'East',
  'TEAM_DIVISION': 'Atlantic',
  'TEAM_NAME': 'Celtics',
  'W': 48},
 1610612739: {'CONF_RANK': 1,
  'DIV_RANK': 1,
  'L': 25,
  'MAX_YEAR': '2016',
  'MIN_YEAR': '1970',
  'PCT': 0.695,
  'SEASON_YEAR': '2015-16',
  'TEAM_ABBREVIATION': 'CLE',
  'TEAM_CITY': 'Cleveland',
  'TEAM_CODE': 'cavaliers',
  'TEAM_CONFERENCE': 'East',
  'TEAM_DIVISION': 'Central',
  'TEAM_NAME': 'Cavaliers',
  'W': 57},
 1610612740: {'CONF_R

In [50]:
G = nx.MultiDiGraph()

In [53]:
G.add_nodes_from(teams_details.to_dict(orient='index').items())

In [54]:
list(G.nodes(data=True))

[(1610612737,
  {'CONF_RANK': 4,
   'DIV_RANK': 2,
   'L': 34,
   'MAX_YEAR': '2016',
   'MIN_YEAR': '1949',
   'PCT': 0.585,
   'SEASON_YEAR': '2015-16',
   'TEAM_ABBREVIATION': 'ATL',
   'TEAM_CITY': 'Atlanta',
   'TEAM_CODE': 'hawks',
   'TEAM_CONFERENCE': 'East',
   'TEAM_DIVISION': 'Southeast',
   'TEAM_NAME': 'Hawks',
   'W': 48}),
 (1610612738,
  {'CONF_RANK': 5,
   'DIV_RANK': 2,
   'L': 34,
   'MAX_YEAR': '2016',
   'MIN_YEAR': '1946',
   'PCT': 0.585,
   'SEASON_YEAR': '2015-16',
   'TEAM_ABBREVIATION': 'BOS',
   'TEAM_CITY': 'Boston',
   'TEAM_CODE': 'celtics',
   'TEAM_CONFERENCE': 'East',
   'TEAM_DIVISION': 'Atlantic',
   'TEAM_NAME': 'Celtics',
   'W': 48}),
 (1610612739,
  {'CONF_RANK': 1,
   'DIV_RANK': 1,
   'L': 25,
   'MAX_YEAR': '2016',
   'MIN_YEAR': '1970',
   'PCT': 0.695,
   'SEASON_YEAR': '2015-16',
   'TEAM_ABBREVIATION': 'CLE',
   'TEAM_CITY': 'Cleveland',
   'TEAM_CODE': 'cavaliers',
   'TEAM_CONFERENCE': 'East',
   'TEAM_DIVISION': 'Central',
   'TEAM_NAME

In [57]:
G.node[1610612738]['TEAM_NAME']

'Celtics'

In [62]:
games().reset_index().pivot_table(index='GAME_ID',
                                  columns='WL',
                                  values='TEAM_ID', 
                                  aggfunc=lambda s: s.iloc[0])

WL,L,W
GAME_ID,,
0021500001,1610612737,1610612765
0021500002,1610612739,1610612741
0021500003,1610612740,1610612744
0021500004,1610612753,1610612764
0021500005,1610612755,1610612738
0021500006,1610612751,1610612741
0021500007,1610612762,1610612765
0021500008,1610612766,1610612748
0021500009,1610612754,1610612761
